In [12]:
import numpy as np
import pandas as pd
import math
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve


def printVar (varname, var):

    print(varname)
    print(var)

def saveArray (name, array):

    pd.DataFrame(array).to_csv("savedVariables/" + name + ".csv")



In [13]:
# Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# Define mesh geometry and boundary conditions

Lx = .01 #width of domain
Ly = .01 #height of domain

fac = 1

u_left = fac*1e-6      #u boundary condition at the left boundary
v_left = fac*1e-6      #v boundary condition at the left boundary

u_right = fac*1e-6     #u boundary condition at the right boundary
v_right = fac*1e-6     #v boundary condition at the right boundary

u_top = fac*1e-6       #u boundary condition at the top boundary
v_top = fac*1e-6       #v boundary condition at the top boundary

u_bottom = fac*1e-6    #u boundary condition at the bottom boundary 
v_bottom = fac*1e-6    #v boundary condition at the bottom boundary

nx = 10            #number of control volumes in the x-direction
ny = 10            #number of control volumes in the y-direction


In [14]:
# # Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# # Define mesh geometry and boundary conditions

# Lx = .01 #width of domain
# Ly = .01 #height of domain

# fac = 1

# u_left = fac*5e-6      #u boundary condition at the left boundary
# v_left = fac*3e-6      #v boundary condition at the left boundary

# u_right = fac*5e-6     #u boundary condition at the right boundary
# v_right = fac*1e-6     #v boundary condition at the right boundary

# u_top = fac*1e-6       #u boundary condition at the top boundary
# v_top = fac*1e-6       #v boundary condition at the top boundary

# u_bottom = fac*7e-6    #u boundary condition at the bottom boundary 
# v_bottom = fac*1e-6    #v boundary condition at the bottom boundary

# nx = 10            #number of control volumes in the x-direction
# ny = 10            #number of control volumes in the y-direction


In [15]:
# Define material properties

rho = 8050         #density of steel in kg/m^3
mu = 78*10e9       #shear modulus in Pa
K = 1.6*10e11      #bulk modulus in Pa

Lambda = K - (2/3)*mu

print(Lambda)

1080000000000.0


In [16]:


## Define required variables

tf = 1              #total time in seconds
dx = Lx/nx          #length of each control volume in the x-direction
dy = Ly/ny          #length of each control volume in the y-direction
dt = tf/100         #size of time steps
Sfx = dy            #area vector x component (Area of East and West Faces)
Sfy = dx            #area vector y component (Area of North and South Faces)


In [17]:
## Define  position and time vectors

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
x = np.zeros((1,nx+2))   #position vector defining the center points of the control volumes in the x-direction.
x[0,nx+1] = Lx
x[0,1:nx+1] = np.arange(dx/2,Lx,dx)

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
y = np.zeros((1,ny+2))   #position vector defining the center points of the control volumes in the y-direction.
y[0,ny+1] = Ly
y[0,1:ny+1] = np.arange(dy/2,Ly,dy)

print(x)
print(y)


[[0.     0.0005 0.0015 0.0025 0.0035 0.0045 0.0055 0.0065 0.0075 0.0085
  0.0095 0.01  ]]
[[0.     0.0005 0.0015 0.0025 0.0035 0.0045 0.0055 0.0065 0.0075 0.0085
  0.0095 0.01  ]]


In [18]:

## Define required displacement matrices and b vector

u_new = np.ones([nx*ny,2])         #unknown displacements at t + 1    (Column 1 = x, Column2 = y)
u_old = np.zeros([nx*ny,2])         #displacement at time t
u_old_old = np.zeros([nx*ny,2])     #displacement at time t - 1

#rhs vector
b_x = np.zeros([nx*ny,1])
b_y = np.zeros([nx*ny,1])

In [19]:
## Define coefficient matrix A with no boundary conditions

A_no_boundary = np.zeros([ny*nx, ny*nx])
A_no_boundary = csr_matrix(A_no_boundary)

# To display A_no_boundary:

print(A_no_boundary.todense().shape)

(100, 100)


## Define coefficient matrix A with no boundary conditions

In [20]:
a_P = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((4*mu+2*Lambda)*Sfx)/dy

a_N = ((2*mu + Lambda)*Sfx)/dx
a_S = ((2*mu + Lambda)*Sfx)/dx
a_E = ((2*mu + Lambda)*Sfx)/dx
a_W = ((2*mu + Lambda)*Sfx)/dx


In [21]:
for j in np.arange(1, ny-1):     #main diagonal, starting from y=1 because all values at y=0 have B.C.s

    for k in np.arange((j*nx)+1, (j*nx)+nx-1):  
        
        A_no_boundary[k,k] = a_P
        A_no_boundary[k,k+nx] = -a_N
        A_no_boundary[k,k-nx] = -a_S
        A_no_boundary[k,k+1] = -a_E
        A_no_boundary[k,k-1] = -a_W

        # print("\tN: " + str(k+nx), "\nW: " + str(k-1), "\tP: " + str(k), "\tE: " + str(k+1), "\n\tS: " + str(k-nx) + "\n")
        
saveArray("A_no_boundary_no_BCs", A_no_boundary.todense())

In [22]:
## Time loop

for time in [1]:

    print("\nTime = " + str(time))
    
    # Set number of maximum iterations for convergence
    
    icorr = 1
    maxcorr = 100

    # Make directory

    !mkdir savedVariables/{time}
    
    # while True:
    for i in [1]:
            
        u_previous = u_new #store latest guess values
        
        # Prepare matrix A and b vector for the x-equation
        
        A_x = A_no_boundary #make copy of matrix A with no boundary conditions
        
        for k in np.arange(0,nx*ny):   # k is the cell number

            #Bottom left corner coefficients
                
            if k == 0:

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2))+((6*mu+3*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #an
                A_x[k,k+nx] = -a_N 

                #ae
                A_x[k,k+1] = -a_E 

                #b_x = A + B(ub) + C(ul) + D(ve + vne - 2vl) + E(vn + vne - 2vb)

                b_x[k] =   ( ((Lambda/(3*dy))+(mu/(3*dx)))*(u_previous[k+nx+1,1] - v_left) - ((mu + Lambda)/dy)*(u_previous[k+nx,0] - 3*u_previous[k,0] + 2*u_bottom ) )*Sfx - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy + 2*u_left*a_W + 2*u_bottom*a_S
                

            #Bottom right corner coefficients
                
            elif  k == nx-1:

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2))+((6*mu+3*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #an
                A_x[k,k+nx] = -a_N 

                #aw
                A_x[k,k-1] = -a_W 

                #b_x = A + B(u_b) + C(u_r) + D(-v_w -v_nw + 2v_r) + E(-v_n -v_nw + 2v_b)

                b_x[k] = ( ((Lambda/(3*dy))+(mu/(3*dx)))*(v_right - u_previous[k+nx-1,1]) - ((mu + Lambda)/dy)*(u_previous[k+nx,0] - 3*u_previous[k,0] + 2*u_bottom ) )*Sfx - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy + 2*u_right*a_E + 2*u_bottom*a_S
                

            # Top Left Corner Coeficients
                
            elif  k == (nx*ny)-(nx):
                
                A_x[k,k] = (rho*dx*dy/(dt**2))+((6*mu+3*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy             
                
                #as
                A_x[k,k-nx] = -a_S

                #ae                
                A_x[k,k+1] = -a_E 

                #b_x = A + B(ut) + C(ul) + D(-ve -vse + 2vl) + E(-vs - vse + 2vt) 
                
                b_x[k] = ( ((Lambda/(3*dy))+(mu/(3*dx)))*(v_left - u_previous[k-nx+1,1]) - ((mu + Lambda)/dy)*(2*u_top - 3*u_previous[k,0] + u_previous[k-nx,0]) )*Sfx - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy + 2*u_left*a_W + 2*u_top*a_N
                
                
            # Top right corner coefficients
                
            elif  k == (nx*ny)-1:

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2))+((6*mu+3*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #as
                A_x[k,k-nx] = -a_S

                #aw
                A_x[k,k-1] = -a_W 

                #b_x = A + B(u_t) + C(u_r) + D(+ v_w + v_sw - 2v_r) + E(+ v_s + v_sw - 2v_t)

                b_x[k] = ( ((Lambda/(3*dy))+(mu/(3*dx)))*(u_previous[k-nx-1,1] - v_right) - ((mu + Lambda)/dy)*(2*u_top - 3*u_previous[k,0] + u_previous[k-nx,0]) )*Sfx - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy + 2*u_right*a_E + 2*u_top*a_N
                
                
            # Center Bottom Boundaries

            elif  (k > 0) & (k < nx):

                # print("x - Bottom Center Coefficents for Cell: " + str(k))

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #an
                A_x[k,k+nx] = -a_N 
                
                #ae
                A_x[k,k+1] = -a_E 

                #aw
                A_x[k,k-1] = -a_W 

                #b_x = A + B(u_b) + C(v_ne - v_nw)

                b_x[k] = ( ((Lambda/(3*dy))+(mu/(4*dx)))*(u_previous[k+nx+1,1] - u_previous[k+nx-1,1])  - ((mu + Lambda)/dy)*(u_previous[k+nx,0] - 3*u_previous[k,0] + 2*u_bottom) )*Sfx - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy + 2*u_bottom*a_S
            
            # Center Top Boundaries

            elif  ( k > (nx*ny)-(nx) ) & ( k < (nx*ny)-1 ):

                # print("x - Top Center Coefficents for Cell: " + str(k))

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #as
                A_x[k,k-nx]  = -a_S
                
                #ae
                A_x[k,k+1] = -a_E 

                #aw
                A_x[k,k-1] = -a_W 

                #b_x = A + B(u_t) + C(-v_se + v_sw)

                b_x[k] = ( ((Lambda/(3*dy))+(mu/(4*dx)))*(u_previous[k-nx-1,1] - u_previous[k-nx+1,1])  - ((mu + Lambda)/dy)*(2*u_top - 3*u_previous[k,0] + u_previous[k-nx,0] ))*Sfx - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy + 2*u_top*a_N
            
            # Center Left Boundaries

            elif  ( k > 0) & ( k%nx == 0 ) & ( k < (nx*ny)-(nx) ):

                # print("x - Left Center Coefficents for Cell: " + str(k))

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #an
                A_x[k,k+nx] = -a_N 
                
                #as
                A_x[k,k-nx] = -a_S

                #ae
                A_x[k,k+1] = -a_E 

                #b_x = A + B(u_l) + C( + v_ne - v_se)

                b_x[k] = ( ((Lambda/(4*dy))+(mu/(3*dx)))*(u_previous[k+nx+1,1] - u_previous[k-nx+1,1]) - ((mu + Lambda)/dy)*(u_previous[k+nx,0] - 2*u_previous[k,0] + u_previous[k-nx,0] ))*Sfx - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy + 2*u_left*a_W
                

            # Center Right Boundaries

            elif  ( k > nx - 1) & ( k%(nx) == nx - 1 ) & ( k < nx*ny -1 ):

                # print("x - Right Center Coefficents for Cell: " + str(k))

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #an
                A_x[k,k+nx] = -a_N 
                
                #as
                A_x[k,k-nx] = -a_S

                #aw
                A_x[k,k-1] = -a_W 

                #b_x = A + B(u_r) + C( - v_nw + v_sw)

                b_x[k] = ( ((Lambda/(4*dy))+(mu/(3*dx)))*(u_previous[k-nx-1,1] - u_previous[k+nx-1,1]) - ((mu + Lambda)/dy)*(u_previous[k+nx,0] - 2*u_previous[k,0] + u_previous[k-nx,0] ))*Sfx - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy + 2*u_right*a_E
                
            
            else:

                b_x[k] = ( ((Lambda/(4*dy))+(mu/(4*dx)))*(u_previous[k+nx+1,1] - u_previous[k-nx+1,1] - u_previous[k+nx-1,1] + u_previous[k-nx-1,1]) - ((mu + Lambda)/dy)*(u_previous[k+nx,0] - 2*u_previous[k,0] + u_previous[k-nx,0]) )*dx*dy - (rho/(dt**2))*(u_old_old[k,0] - 2*u_old[k,0])*dx*dy

        saveArray(str(time) + "/A_x_" + str(icorr) , A_x.todense())   
        saveArray(str(time) + "/b_x_" + str(icorr), b_x)      
        
        ## Solve the x-equation
        
        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)

        saveArray(str(time) + "/Ux_" + str(icorr), Ux)
        
        ## Prepare matrix A and b vector for the y-equation
        
        A_y = A_no_boundary #make copy of A with no boundary conditions

        
        for k in np.arange(0,nx*ny):   # k is the cell number
            
            #Bottom left corner coefficients
            
            if k == 0:
                
                A_y[k,k] = (rho*dx*dy/(dt**2))+((6*mu+3*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy      
                
                #an
                A_y[k,k+nx] =  -a_N 

                #ae                
                A_y[k,k+1] = -a_E  

                # b_y = A + B(vb) + C(vl) + D(ue + une - 2ul) + E(un + une - 2ub) 
                
                b_y[k] = ( ((Lambda/(3*dy))+(mu/(3*dx)))*(u_previous[k+nx+1,0] - u_left) - ((mu + Lambda)/dx)*(u_previous[k+1,1] - 3*u_previous[k,1] + 2*v_left ) )*Sfx - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy + 2*v_left*a_W + 2*v_bottom*a_S
                
            #Bottom right corner coefficients

            elif k == nx-1:
                
                A_y[k,k] = (rho*dx*dy/(dt**2))+((6*mu+3*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy 
                
                #an
                A_y[k,k+nx] =  -a_N 

                #aw                
                A_y[k,k-1] = -a_W  

                #b_y = A + B(vb) + C(vr) + D(- u_w - u_nw + 2u_r) + E(- u_n - u_nw + 2u_b) 
                
                b_y[k] = ( ((Lambda/(3*dy))+(mu/(3*dx)))*(u_right - u_previous[k+nx-1,0]) - ((mu + Lambda)/dx)*(u_previous[k-1,1] - 3*u_previous[k,1] + 2*v_right ) )*Sfx - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy + 2*v_right*a_E + 2*v_bottom*a_S
                
                     
            #Top left corner coefficients
                
            elif k == (nx*ny)-(nx):
                
                A_y[k,k] = (rho*dx*dy/(dt**2))+((6*mu+3*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy        
                
                #as
                A_y[k,k-nx] = -a_S 

                #ae                
                A_y[k,k+1] = -a_E  
                
                b_y[k] = ( ((Lambda/(3*dy))+(mu/(3*dx)))*(u_left - u_previous[k-nx+1,0]) - ((mu + Lambda)/dy)*(2*v_left - 3*u_previous[k,1] + u_previous[k+1,1]) )*Sfx - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy + 2*v_left*a_W + 2*v_top*a_N
                
                
            #Top right corner coefficients
                
            elif k == (nx*ny)-1:
                #ap
                
                A_y[k,k] = (rho*dx*dy/(dt**2))+((6*mu+3*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy
                
                #as
                A_y[k,k-nx] = -a_S 

                #aw                
                A_y[k,k-1] = -a_W  

                #b_y = A + B(v_t) + C(v_r) + D(+ u_w + u_sw - 2u_r) + E(+ u_s + u_sw - 2u_t) 
                
                b_y[k] =  ( ((Lambda/(3*dy))+(mu/(3*dx)))*(u_previous[k-nx-1,0] - u_right) - ((mu + Lambda)/dy)*(2*v_right - 3*u_previous[k,1] + u_previous[k-1,1]) )*Sfx - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy + 2*v_right*a_E + 2*v_top*a_N
                
            
            # Center Bottom Boundaries

            elif (k > 0) & (k < nx):

                # print("y - Bottom Center Coefficents for Cell: " + str(k))

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #an
                A_y[k,k+nx] =  -a_N 
                
                #ae
                A_y[k,k+1] = -a_E  

                #aw
                A_y[k,k-1] = -a_W  

                #b_y = A + B(v_b) + C(+ u_ne - u_nw)

                b_y[k] = ( ((Lambda/(4*dx))+(mu/(3*dy)))*(u_previous[k+nx+1,0] - u_previous[k+nx-1,0]) - ((mu + Lambda)/dx)*(u_previous[k+1,1] - 3*u_previous[k,1] + u_previous[k-1,1]) )*Sfx - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy + 2*v_bottom*a_S
                

            # Center Top Boundaries

            elif ( k > (nx*ny)-(nx )) & ( k < (nx*ny)-1 ) :

                # print("y - Top Center Coefficents for Cell: " + str(k))

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #as
                A_y[k,k-nx] = -a_S 
                
                #ae
                A_y[k,k+1] = -a_E  

                #aw
                A_y[k,k-1] = -a_W  

                #b_y = A + B(v_t) + C(- u_se + u_sw)

                b_y[k] = ( ((Lambda/(4*dx))+(mu/(3*dy)))*(u_previous[k-nx-1,0] - u_previous[k-nx+1,0]) - ((mu + Lambda)/dx)*(u_previous[k+1,1] - 2*u_previous[k,1] + u_previous[k-1,1] ))*Sfx - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy + 2*v_top*a_N
                

            # Center Left Boundaries

            elif ( k > 0) & ( k%nx == 0 ) & ( k < (nx*ny)-(nx) ):

                # print("x - Left Center Coefficents for Cell: " + str(k))

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #an
                A_y[k,k+nx] =  -a_N 
                
                #as
                A_y[k,k-nx] = -a_S 

                #ae
                A_y[k,k+1] = -a_E  

                #b_y = A + B(v_l) + C( + u_ne - u_se)

                b_y[k] = ( ((Lambda/(3*dx))+(mu/(4*dy)))*(u_previous[k+nx+1,0] - u_previous[k-nx+1,0]) - ((mu + Lambda)/dx)*(u_previous[k+1,1] - 3*u_previous[k,1] + 2*v_left ))*Sfx - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy + 2*v_left*a_W
                

            # Center Right Boundaries

            elif ( k > nx - 1) & ( k%(nx) == nx - 1 ) & ( k < nx*ny -1 ):

                # print("x - Right Center Coefficents for Cell: " + str(k))

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((6*mu+3*Lambda)*Sfy)/dy

                #an
                A_y[k,k+nx] =  -a_N 
                
                #as
                A_y[k,k-nx] = -a_S 

                #aw
                A_y[k,k-1] = -a_W  

                #b_y = A + B(v_r) + C( - u_nw + u_sw)

                b_y[k] = ( ((Lambda/(3*dx))+(mu/(4*dy)))*(u_previous[k-nx-1,0] - u_previous[k+nx-1,0]) - ((mu + Lambda)/dx)*(2*v_right - 3*u_previous[k,1] + u_previous[k-1,1] ))*Sfx - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy + 2*v_right*a_E
                

            else:

                b_y[k] =  ( ((Lambda/(4*dx))+(mu/(4*dy)))*(u_previous[k+nx+1,0] - u_previous[k-nx+1,0] - u_previous[k+nx-1,0] + u_previous[k-nx-1,0]) - ((mu + Lambda)/dx)*(u_previous[k+1,1] - 2*u_previous[k,1] + u_previous[k-1,1]) )*dx*dy - (rho/(dt**2))*(u_old_old[k,1] - 2*u_old[k,1])*dx*dy
     
        saveArray(str(time) + "/A_y_" + str(icorr) , A_y.todense())   
        saveArray(str(time) + "/b_y_" + str(icorr), b_y)
        
        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)

        saveArray(str(time) + "/Uy_" + str(icorr), Uy)
        
        # ## Update u_new with new x and y displacements

        u_new1 = np.vstack((Ux, Uy)).T
        saveArray (str(time) + "/u_new1_" + str(icorr), u_new1)    
        saveArray (str(time) + "/u_previous_" + str(icorr), u_previous)

        # # u_new[:,0] = Ux
        # # u_new[:,1] = Uy
        
        ## Calculate the residual of each iteration
        
        normFactor = np.max(u_new1)
        residual = math.sqrt(np.mean((u_new1 - u_previous)**2))/normFactor
        tolerance = 1e-6

        u_new = u_new1
    
        # #Print out residual of each iteration
        print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}".format(icorr, residual, normFactor))
            
        ## Convergence check
        
        if residual < tolerance:
            
            break
            
        elif icorr > maxcorr:
            
            break
            
        
        icorr = icorr + 1
    
    ## Update variables
    
    u_old_old = u_old
    u_old = u_new
    
    ## Add the boundary conditions to the solution matrices
    u_reshape_x = u_new[:,0].reshape(nx, ny)
    u_reshape_y = u_new[:,1].reshape(nx, ny)
    
    #Boundary conditions for u matrix
    U = np.zeros((nx+2,ny+2))
    U[0,1:nx+1] = u_bottom
    U[1:ny+1,0] = u_left
    U[1:ny+1,nx+1] = u_right
    U[ny+1,1:nx+1] = u_top    
    U[1:nx+1,1:nx+1] = u_reshape_x
    
    # #Boundary conditions for v matrix
    V = np.zeros((nx+2,ny+2))
    V[0,1:nx+1] = v_bottom
    V[1:ny+1,0] = v_left
    V[1:ny+1,nx+1] = v_right
    V[ny+1,1:nx+1] = v_top    
    V[1:nx+1,1:nx+1] = u_reshape_y  
    
    #Modulus of displacement
    disp_mod = np.sqrt((U**2) + (V**2))
    disp_mod_converted = disp_mod*1e6 #Convert displacement to micrometers
    
    # ## Plot the results

    import plotly.graph_objects as go

    disp_mod = np.sqrt((U**2) + (V**2))*1e-6

    dispx, dispy = np.meshgrid(x, y)
    fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])
    
fig.show()   
    






Time = 1
mkdir: cannot create directory ‘savedVariables/1’: File exists
icorr = 1, Residual = 0.95193088309739015695, normFactor = 0.66043007866841729747


ModuleNotFoundError: No module named 'plotly'